In [11]:
# Modified from
# http://dpmartin42.github.io/posts/r/imbalanced-classes-part-1
# http://dpmartin42.github.io/posts/r/imbalanced-classes-part-2


#Class1 == 0
#Calss2 == 1

library(pROC) # for ROC AUC calculations
library(PRROC) # for PR AUC calculations
library(caret)

aucSummary <- function(data, lev = NULL, model = NULL){
  
  index_class2 <- data$obs == "Class2"
  index_class1 <- data$obs == "Class1"
  
  #calc the auc metrics
  pr <- pr.curve(data$Class2[index_class2],
                 data$Class2[index_class1],
                 curve = FALSE)
  
  roc <- roc.curve(data$Class2[index_class2],
                   data$Class2[index_class1],
                   curve = FALSE)
  
  pr_out <- pr$auc.integral
  
  roc_out <- roc$auc
  
  obs <- data$obs
  
  
  #Assign threshold
  pre <- ifelse(data$Class2 > .5, 'Class2', 'Class1')
  
  #Calculate Accuracy  
  acc <- mean(obs == pre)
  
  out <- c(pr_out,roc_out,acc)  
  
  names(out) <-c("AUPRC","AUROC","Accuracy")
  
  out  
}

In [12]:
train <- twoClassSim(500,
                           intercept = 0,
                           linearVars = 10,
                           noiseVars = 5)

In [14]:
ctrl <- trainControl(method = "repeatedcv",
                     number = 4,
                     repeats = 2,
                     summaryFunction = aucSummary,
                     classProbs = TRUE)

fit <- train(Class ~ .,
                  data = train,
                  method = "rf",
                  verbose = FALSE,
                  trControl = ctrl,
                  metric = "AUPRC",
                  parallel= TRUE)

fit

Random Forest 

500 samples
 20 predictor
  2 classes: 'Class1', 'Class2' 

No pre-processing
Resampling: Cross-Validated (4 fold, repeated 2 times) 
Summary of sample sizes: 375, 375, 375, 375, 375, 375, ... 
Resampling results across tuning parameters:

  mtry  AUPRC      AUROC      Accuracy
   2    0.9648250  0.9205313  0.775   
  11    0.9649962  0.9217022  0.849   
  20    0.9610269  0.9139089  0.844   

AUPRC was used to select the optimal model using  the largest value.
The final value used for the model was mtry = 11.